# **Series temporales multivariadas con Auto ARIMA**

In [ ]:
from tabulate import tabulate

def tabl_prt(to_primt):
    print(tabulate(to_primt, headers='keys'),'\n')

## **Carga y análisis de datos**

In [ ]:
import pandas as pd

df = pd.read_csv('../../data/ARIMA/energy_consumption.csv')

tabl_prt(df.describe(include='all'))

tabl_prt(df.head())
tabl_prt(df.tail())

- **timeStamp**: Marca de tiempo para cada registro (formato: YYYY-MM-DD HH:MM:SS).
- **demand**: Consumo horario de energía en kWh. Este valor fluctúa según patrones diarios y estacionales.
- **precip**: Precipitación en mm. Este valor está correlacionado negativamente con la demanda (mayor precipitación puede reducir el consumo de energía).
- **temp**: Temperatura en grados Celsius. Este valor está correlacionado positivamente o negativamente dependiendo de si hace calor (mayor uso de aire acondicionado) o frío (mayor uso de calefacción).

In [ ]:
# Convertir la columna de marca de tiempo

df['timeStamp']=pd.to_datetime(df['timeStamp'])

In [ ]:
# Trazar la columna de demanda
import plotly.express as px

fig = px.line(df, x='timeStamp', y='demand', title='Energy Consumption')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(step="all")
        ])
    )
)
fig.show()

De la salida anterior, el conjunto de datos tiene estacionalidad (ciclos repetitivos).

- Dado que el conjunto de datos tiene estacionalidad, podemos decir que no es estacionario.

Pero aún así, debemos realizar una verificación estadística utilizando la prueba de Dickey-Fuller (ADF) aumentada para evaluar la estacionariedad en nuestro conjunto de datos.

- Si encontramos que el conjunto de datos no es estacionario después de la prueba de ADF, tendremos que realizar diferencias para hacerlo estacionarios.

- Auto Arima realiza la diferencia automáticamente.

> El siguiente paso es establecer la marca de tiempo como la columna de índice.



In [ ]:
el_df=df.set_index('timeStamp')

### Graficando subtramas
Las subtramas mostrarán las variables dependientes del tiempo en el conjunto de datos.

Visualizaremos las columnas de demand, precip y temp.

In [ ]:
el_df.plot(subplots=True)


### Verificar valores faltantes o nulos

In [ ]:
print ("\nMissing values :  ", df.isnull().any())

Imputar valores nulos

In [ ]:
df['demand']=df['demand'].ffill()

df['temp']=df['temp'].ffill()

df['temp']=df['precip'].ffill()

In [ ]:
print ("\nMissing values :  ", df.isnull().any())

### Dataset resampling
La serie temporal tiene muchos puntos que pueden ser difíciles de analizar y visualizar.
- Necesitamos volver a muestrear el tiempo comprimiéndolo y agregándolo a intervalos mensuales.
- Tendremos menos puntos de datos que sean más fáciles de analizar.

El método `resample()` agregará todos los puntos de datos en las series de tiempo y los cambiará a intervalos mensuales.

In [ ]:
el_df.resample('ME').mean()

### Graficando los nuevos subplots

In [ ]:
el_df.resample('ME').mean().plot(subplots=True)

In [ ]:
# Guardar el conjunto de datos resmuestrado

final_df=el_df.resample('ME').mean()

## **Implementación del modelo Auto ARIMA**

Implementamos el modelo Auto ARIMA utilizando la librería **pmdarima Time-Series**. 

Proporciona la función `auto_arima()` que genera automáticamente los valores de parámetros óptimos.

En la función `auto_arima ()` pasamos el `final_df`, que es nuestro conjunto de datos remuestreado. 

- Seleccionamos la columna de demanda ya que esto es lo que el modelo quiere predecir.
- La función puede usar búsqueda de cuadrícula o búsqueda aleatoria para encontrar los valores de parámetros óptimos.
- Intenta múltiples combinaciones de P, D y Q y luego selecciona las óptimas.

La función `auto_arima ()` también tiene los siguientes parámetros:
- `m=12`: Representa el número de meses en un año.

- `start_p=0`: Representa el valor p mínimo que la función puede seleccionar durante la búsqueda aleatoria.

- `start_q=0`: Representa el valor q mínimo que la función puede seleccionar durante la búsqueda aleatoria.

- `max_order=4`: representaLosValoresMáximosDeP,DYQQueElModeloPuedeSeleccionarDuranteLaBúsquedaAleatoria

- `test='adf'`: Es una prueba de Dickey-Fuller Aumentada (ADF) para verificar la estacionariedad en nuestro conjunto de datos. 
    - Si el conjunto de datos no es estacionario después de la prueba ADF, la función `auto_arima()` generará automáticamente el valor **d** para la diferenciación.
    - Si el conjunto de datos es estacionario, establece **d = 0** (no es necesario diferenciar).

- `suppress_warnings=True`: Ignora las advertencias durante la búsqueda de parámetros.

- `stepwise=True`: Ejecutará la búsqueda aleatoria para encontrar los parámetros óptimos. La búsqueda en la cuadrícula es más exhaustiva ya que intenta todas las combinaciones de parámetros, pero es lenta.

In [ ]:
import pmdarima as pm

model = pm.auto_arima(final_df['demand'],
                      m=12, seasonal=True,
                      start_p=0, start_q=0, max_order=4, test='adf', error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True, trace=True)

De la salida anterior, el mejor modelo es ARIMA (1,0,1) (p = 1, d = 0 y q = 1). La función establece automáticamente d = 0 porque la prueba ADF encontró que el conjunto de datos es estacionario.
- Anteriormente habíamos observado los gráficos del conjunto de datos de la serie temporal para tener estacionalidad.
- Por lo tanto, pensamos que la serie temporal no era estacionaria, de ahí una necesidad de diferenciar.

Pero el uso de la prueba ADF, que es una prueba estadística, descubrió que la estacionalidad es insignificante. 
- La prueba ADF es más precisa que observar/visualizar las parcelas. 
- Es por eso que la función establece d = 0, y no hay necesidad de diferenciar.

Después de inicializar la función `auto_arima ()`, el siguiente paso es dividir el conjunto de datos de la serie temporal.

## **Ajustar el modelo**

In [ ]:
train = final_df[(final_df.index.get_level_values(0) >= '2012-01-31')
                 & (final_df.index.get_level_values(0) <= '2014-04-30')]

train.tail()

El código selecciona los puntos de datos del 2012-01-31 a 2014-04-30 para el entrenamiento modelo. 

Obtenemos los puntos de datos para la prueba del modelo utilizando el siguiente código:

In [ ]:
test = final_df[(final_df.index.get_level_values(0) > '2014-04-30')]

test

### Ajustar el modelo


In [ ]:
model.fit(train['demand'])

### Usar el modelo para hacer predicciones


In [ ]:
forecast = model.predict(n_periods=8, return_conf_int=True)

`n_periods=8`: Representa el número de puntos de datos en el marco de datos de prueba que predecirá el modelo.

In [ ]:
# Predicciones
forecast_df = pd.DataFrame(forecast[0], index=test.index, columns=['Prediction'])

forecast_df

In [ ]:
# Graficando

import matplotlib.pyplot as plt
pd.concat([final_df['demand'],forecast_df],axis=1).plot()


From the line chart above:

- The blue line is the actual energy demand.
- The orange line is the predicted energy demand.
- The Auto ARIMA model has performed well and has made accurate predictions. The blue and orange lines are close to each other.

We can now use this model to predict unseen future values.

Del gráfico anterior:

- La línea azul es la demanda de energía real.
- La línea naranja es la demanda de energía predicha.
- El modelo Auto ARIMA se ha desempeñado bien y ha hecho predicciones precisas: Las líneas azules y naranjas están cerca unas de la otra.

Ahora podemos usar este modelo para predecir valores futuros invisibles.

In [ ]:
forecast1=model.predict(n_periods=8, return_conf_int=True)
forecast_range=pd.date_range(start='2014-05-31', periods=8,freq='ME')

`n_periods=8`: Representa el número de puntos de datos que el modelo predecirá en el futuro. Las fechas futuras son de 2014-05-31. 

In [ ]:
forecast1_df = pd.DataFrame(forecast1[0], index=forecast_range, columns=['Prediction'])

forecast1_df

In [ ]:
# Graficando las predicciones
pd.concat([final_df['demand'],forecast1_df],axis=1).plot()
